In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import firedrake
import icepack, icepack.models, icepack.plot

# Synthetic ice sheet

In this demo we'll run an experiment inspired by the paper [*Fjord insertion into continental margins
driven by topographic steering of ice*](https://www.nature.com/articles/ngeo201) by Kessler and others.
This work simulated the evolution of an entire ice sheet on millenial timescales, with the added twist that the bedrock topography could evolve under the influence of erosion.

### Geometry

The model domain is a circle 250 km wide, but with a 1km-high ridge at a radius of 200 km.
This ridge is punctuated by four valleys of varying depths from which ice streams form.

In [ ]:
import pygmsh

R = 250e3
δx = 15e3

geometry = pygmsh.built_in.Geometry()

points = [geometry.add_point([+R, 0, 0], lcar=δx),
          geometry.add_point([0, +R, 0], lcar=δx),
          geometry.add_point([-R, 0, 0], lcar=δx),
          geometry.add_point([0, -R, 0], lcar=δx)]
center = geometry.add_point([0, 0, 0], lcar=δx)

arcs = [geometry.add_circle_arc(points[k], center, points[(k + 1) % len(points)])
        for k in range(len(points))]

line_loop = geometry.add_line_loop(arcs)
plane_surface = geometry.add_plane_surface(line_loop)

physical_lines = [geometry.add_physical(arc) for arc in arcs]
physical_surface = geometry.add_physical(plane_surface)

Now we're going to do some fancy tricks to make the mesh more refined around the mountain range.
First, we'll make some circular arcs around the radius of 200km.
We won't put these arcs in any physical entity, so they won't get included in the output geometry; they're just there as a cue for later geometric constructions.

In [ ]:
r = 125e3
cpoints = [geometry.add_point([+r, 0, 0], lcar=δx),
           geometry.add_point([0, +r, 0], lcar=δx),
           geometry.add_point([-r, 0, 0], lcar=δx),
           geometry.add_point([0, -r, 0], lcar=δx)]

carcs = [geometry.add_circle_arc(cpoints[k], center, cpoints[(k + 1) % len(cpoints)])
         for k in range(len(cpoints))]

Then we'll add a bit of text to the end of the code for the geometry that creates field objects.
The value of this field transitions from the characteristic length of 15km throughout most of the domain down to 1km in the neighborhood of the mountain range.
Finally, we'll set this as the background field determining the mesh's characteristic length scale and turn off all the other built-in methods for setting the characteristic length.

In [ ]:
code = geometry.get_code()

code += """
lc = {lc:};

Field[1] = Distance;
Field[1].NNodesByEdge = 100;
Field[1].EdgesList = {{{edges:}}};

Field[2] = Threshold;
Field[2].IField = 1;
Field[2].LcMin = lc / {shrink_factor:};
Field[2].LcMax = lc;
Field[2].DistMin = {dist_min:};
Field[2].DistMax = {dist_max:};

Background Field = 2;

Mesh.CharacteristicLengthExtendFromBoundary = 0;
Mesh.CharacteristicLengthFromPoints = 0;
Mesh.CharacteristicLengthFromCurvature = 0;
""".format(edges=', '.join([arc.id for arc in carcs]), lc=δx, shrink_factor=5.,
           dist_min=40e3, dist_max=60e3)

with open('ice-sheet.geo', 'w') as geo_file:
    geo_file.write(code)

You can see the result by plotting the mesh -- everything is much more refined around the mountain range.

In [ ]:
!gmsh -2 -format msh2 -o ice-sheet.msh ice-sheet.geo

In [ ]:
mesh = firedrake.Mesh('ice-sheet.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh)

In [ ]:
from firedrake import exp, sqrt
x, y = firedrake.SpatialCoordinate(mesh)

def tanh(z):
    return (exp(z) - exp(-z)) / (exp(z) + exp(-z))

def θ(z):
    return (tanh(z) + 1) / 2

def sech(z):
    return 2 / (exp(z) + exp(-z))

b0 = firedrake.Constant(400)
h0 = firedrake.Constant(1000)

a = firedrake.Constant(50e3)
ξ = (sqrt(x**2 + y**2) - r) / a

ρ1 = firedrake.Constant(1/4)
μ1 = 1 - ρ1 * θ(3 * (x - r/4) / a) * sech(2 * y / a)

ρ2 = firedrake.Constant(3/8)
μ2 = 1 - ρ2 * θ(3 * (y - r/4) / a) * sech(2 * x / a)

ρ3 = firedrake.Constant(1/2)
μ3 = 1 - ρ3 * θ(3 * (-x + r/4) / a) * sech(2 * y / a)

ρ4 = firedrake.Constant(5/8)
μ4 = 1 - ρ4 * θ(3 * (-y + r/4) / a) * sech(2 * x / a)

b_expr = b0 * (1 - θ(3 * ξ)) + (h0 - b0) * sech(2 * ξ) * μ1 * μ2 * μ3 * μ4

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
b = firedrake.interpolate(b_expr, Q)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(b, 40, cmap='magma', axes=axes)
fig.colorbar(contours)

In [ ]:
firedrake.plot(b, cmap='magma', plot3d=True)